In [1]:
import sys
import os
from collections import defaultdict
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("./Untitled.ipynb"), '..')))
import numpy as np
from src.bopf.bopf import BagOfPatternFeature
from src.utils import sort_trim_arr
from src.bopf.classifier import classify, classify2
import matplotlib.pyplot as plt
import time
import pandas as pd
from scipy.stats import f_oneway
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import itertools
%matplotlib inline

from src.bopf.classifier import classify, classify2
from src.utils import sort_trim_arr

In [2]:
from src.bopf.bopf_finder import bopf_param_finder_mp, bopf_best_classifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
import pandas as pd

import multiprocessing as mp
from collections import defaultdict
from src.bopf.bopf import BagOfPatternFeature
from src.bopf.classifier import classify, classify2
from src.utils import sort_trim_arr
import numpy as np
import time
from sklearn.metrics import balanced_accuracy_score

In [3]:
main_path = os.path.abspath(os.path.join(os.path.dirname("./Untitled.ipynb"), '..'))

In [4]:
data_path = os.path.join(main_path, "data", "plasticc_subsets", "ddf_wdf_split_simplified", "60_40_split")

In [17]:
train_base = "wdf_train_%s_size1321.npy"
test_base = "wdf_test_%s_size1321.npy"
wd = 3
wl = 100

bopf = BagOfPatternFeature(special_character=True, strategy="special1", test=False)
bopf.load_dataset(data_path, fmt="npy", base=train_base)

bopf.cumsum()

In [18]:
%%time
bopf.bop(wd, wl, verbose=True, window_type="time")

TOTAL DE SEGMENTOS VACIOS:  11271
TOTAL DE TIMESERIES DESCARTADAS:  2
TOTAL DE ts < 6: 0
CPU times: user 1.1 s, sys: 1 µs, total: 1.1 s
Wall time: 1.1 s


In [19]:
len(bopf.valid_train_bop), bopf.valid_train_bop.sum()

(793, 791)

In [20]:
bopf.adjust_label_set()

In [21]:
%%time
bopf.simple_vectors()

CPU times: user 20.9 ms, sys: 2 µs, total: 20.9 ms
Wall time: 18.8 ms


In [11]:
np.sum(bopf.crossL2), np.sum(bopf.crossL)

(4054.2607041280535, 67.22880294019002)

In [19]:
np.sum(bopf.crossL2), np.sum(bopf.crossL)

(4054.2607041280535, 67.22880294019002)

In [22]:
bopf_t = BagOfPatternFeature(special_character=True, strategy="special1", test=True)
bopf_t.load_dataset(data_path, fmt="npy", base=test_base)
bopf_t.cumsum()

bopf_t.bop(wd, wl, verbose=True, window_type="time")

TOTAL DE SEGMENTOS VACIOS:  7502
TOTAL DE TIMESERIES DESCARTADAS:  0
TOTAL DE ts < 6: 0


In [23]:
%%time
predicted_label = classify2_v2(bopf_t.train_bop, bopf.crossL2, bopf.tlabel,
                                bopf_t.m, bopf.c, bopf.bopsize)

CPU times: user 1.31 s, sys: 0 ns, total: 1.31 s
Wall time: 1.31 s


In [28]:
real_label = bopf_t.labels
balanced_accuracy_score(real_label, predicted_label)

0.21631040717669722

In [159]:
bopf.anova(verbose=False)

In [160]:
bopf.fea_num, bopf.bopsize

(0, 256)

In [161]:
bopf.anova_sort()

In [162]:
bopf.sort_trim_arr(verbose=False)

In [163]:
bopf.crossVL(verbose=True)

In [164]:
_ = bopf.crossVL2()

In [165]:
print(wd, wl, "crossVL acc:", bopf.best_score, "crossVL2 acc:", bopf.best2_score, ", dropped ts: ", len(bopf.dropped_ts))

3 10 crossVL acc: 0.22841426695680775 crossVL2 acc: 0.2967653840195415 , dropped ts:  85


In [167]:
bopf_t = BagOfPatternFeature(special_character=True, strategy="special1", test=True)
bopf_t.load_dataset(data_path, fmt="npy", base=test_base)
bopf_t.cumsum()

In [168]:
bopf_t.bop(wd, wl, verbose=True, window_type="time")

TOTAL DE SEGMENTOS VACIOS:  3254
TOTAL DE TIMESERIES DESCARTADAS:  47
TOTAL DE ts < 6: 0


In [169]:
bop_feature_index = bopf.sort_index[:bopf.best2_idx]
bop_fea_num = bopf.best2_idx
test_bop_sort = sort_trim_arr(bopf_t.train_bop, bop_feature_index,
                                  bopf_t.m, bop_fea_num)

In [170]:
bopf_t.train_bop.shape

(135169,)

In [171]:
test_bop_sort.shape

(24288,)

In [172]:
bop_fea_num, bopf_t.bopsize

(45, 256)

In [14]:
import pdb
def classify2_v2(test_bop, tf_idfs, tlabel, mt, c, fea_num):
    res = np.zeros(mt, dtype=int)
    for i in range(mt):
#         if i == 2:
#             pdb.set_trace()
        rmax = -np.inf
        label = -1
        for j in range(c):
            r1 = 0.0
            r2 = 0.0
            r3 = 0.0
            for k in range(fea_num):
                q = test_bop[i + k * mt]
                if q > 0:
#                     if i == 8:
#                         pdb.set_trace()
                    q = 1 + np.log10(q)
                s = tf_idfs[j + k * c]
                r1 += q*s
                r2 += q*q
                r3 += s*s

#             if r2 != 0 and r3 != 0:
#                 dist = r1*r1 / (r2*r3)
#             else:
#                 dist = 0
            dist = r1*r1 / (r2*r3)
            if dist > rmax:
                rmax = dist
                label = tlabel[j]
        res[i] = label
    return res

In [174]:
bop_features = bopf.crossL2[:bopf.c * bopf.best2_idx]
predicted_label = classify2_v2(test_bop_sort, bop_features, bopf.tlabel,
                                bopf_t.m, bopf.c, bop_fea_num)

<ipython-input-173-bfa1dc34c6c9>:28: RuntimeWarning: invalid value encountered in double_scalars
  dist = r1*r1 / (r2*r3)


In [175]:
real_label = np.array(bopf_t.labels)

In [176]:
pred = []
real = []
for i in range(bopf_t.m):
    if bopf_t.valid_train_bop[i]:
        pred.append(predicted_label[i])
        real.append(real_label[i])

In [177]:
balanced_accuracy_score(real, pred)

0.14792390199148647

In [178]:
balanced_accuracy_score(real_label, predicted_label)

0.1368195930026016

In [123]:
bop_feature_index

array([19, 16,  3,  8,  6, 13,  9, 12,  7,  2])

In [124]:
for i in range(bopf_t.m):
    if predicted_label[i] == -1 and bopf_t.valid_train_bop[i]:
        s = 0
        for k in range(bop_fea_num):
            s += test_bop_sort[i + k * bopf_t.m]
        print(i, s)

34 0.0
55 0.0
77 0.0
108 0.0
127 0.0
139 0.0
146 0.0
184 0.0
198 0.0
242 0.0
249 0.0
261 0.0
274 0.0
279 0.0
281 0.0
330 0.0
340 0.0
346 0.0
350 0.0
388 0.0
434 0.0
444 0.0
457 0.0
459 0.0
480 0.0
492 0.0
501 0.0
506 0.0


In [64]:
predicted_label

array([62, 16, -1, -1, 90, 42, 65, 42, -1, 90, 16, 88, 42, 16, 90, 62, 16,
       -1, 42, 90, -1, -1, 42, 65, 65, 90, 65, 42, 16, 88, 42, 16, 16, 42,
       90, 42, -1, -1, 88, 62, -1, 62, -1, 90, 90, 62, 65, 62, 88, 65, 62,
       62, 16, -1, 88, -1, 62, 62, 42, 42, 65, 42, 65, 62, 90, 62, -1, 90,
       16, 42, 62, 62, 16, -1, -1, 88, -1, 88, 16, 65, 42, 42, 42, 65, 42,
       90, 42, -1, 16, 90, 16, 65, -1, 42, -1, 42, -1, 90, 62, -1, 90, 90,
       -1, 88, 90, 62, -1, 42, -1, 62, 16, -1, 62, 90, -1, 62, 16, 42, 90,
       65, 42, 16, 65, 88, 16, -1, 42, 62, 90, 16, -1, 16, -1, 16, 65, -1,
       -1, 65, -1, 90, 42, 90, 90, 42, 16, 42, 62, 88, 88, 62, 90, 65, 90,
       -1, -1, 42, -1, 42, 62, 16, 62, 62, 90, 88, 90, 65, 16, 88, 42, -1,
       90, -1, 16, -1, 16, 42, -1, 88, -1, 16, 42, -1, 16, 90, 65, -1, 90,
       16, -1, 42, 65, 65, 90, 90, -1, -1, 88, 90, -1, 90, 62, 42, 65, -1,
       -1, 16, 65, 90, -1, 65, 88, 65, 62, 42, 90, 42, 90, 16, 42, -1, 16,
       65, 16, 90, 90, 88

In [70]:
i = 3
s = 0
for k in range(bop_fea_num):
    s += test_bop_sort[i + k * bopf_t.m]
s

0.0

In [179]:
a = [1, 2, 3, 4, 5, 6, 7]
b = [25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350]
len(a) * len(b)

98

In [183]:

from multiprocessing import Lock, Process, Queue, current_process
import time
import queue # imported for using queue.Empty exception


def do_job(tasks_to_accomplish, tasks_that_are_done):
    while True:
        try:
            '''
                try to get task from the queue. get_nowait() function will 
                raise queue.Empty exception if the queue is empty. 
                queue(False) function would do the same task also.
            '''
            task, wd, wl = tasks_to_accomplish.get_nowait()
        except queue.Empty:

            break
        else:
            '''
                if no exception has been raised, add the task completion 
                message to task_that_are_done queue
            '''
            print(task)
            tasks_that_are_done.put(task + ' is done by ' + current_process().name + ', wd: ' +  str(wd) + ', wl: ' + str(wl))
            time.sleep(.5)
    return True


def main():
    number_of_task = 10
    number_of_processes = 4
    tasks_to_accomplish = Queue()
    tasks_that_are_done = Queue()
    processes = []

    for i in range(number_of_task):
        tasks_to_accomplish.put(("Task no " + str(i), 2, i))

    # creating processes
    for w in range(number_of_processes):
        p = Process(target=do_job, args=(tasks_to_accomplish, tasks_that_are_done))
        processes.append(p)
        p.start()

    # completing process
    for p in processes:
        p.join()

    # print the output
    while not tasks_that_are_done.empty():
        print(tasks_that_are_done.get())

    return True

In [184]:
if __name__ == '__main__':
    main()

Task no 0
Task no 1
Task no 2
Task no 3
Task no 4
Task no 5
Task no 6
Task no 7
Task no 8
Task no 9
Task no 0 is done by Process-5, wd: 2, wl: 0
Task no 1 is done by Process-6, wd: 2, wl: 1
Task no 2 is done by Process-7, wd: 2, wl: 2
Task no 3 is done by Process-8, wd: 2, wl: 3
Task no 4 is done by Process-5, wd: 2, wl: 4
Task no 5 is done by Process-6, wd: 2, wl: 5
Task no 6 is done by Process-7, wd: 2, wl: 6
Task no 7 is done by Process-8, wd: 2, wl: 7
Task no 8 is done by Process-5, wd: 2, wl: 8
Task no 9 is done by Process-6, wd: 2, wl: 9
